In [61]:
def Jaccard (list1, list2):
    set1, set2 = set(list1), set (list2)

    # Calculate the Jaccard similarity
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_similarity = intersection / union

    return jaccard_similarity


def subgr (list1, list2):
    set1, set2 = set(list1), set (list2)
    intersection = set1.intersection(set2)
    
    p = max ( len(intersection) / len(set1) , len(intersection)/ len(set2)  ) 
    
    return p


def add_v_clone ( tool, clone_dict, v ):
    for k in range ( len (clone_dict[tool]) ):
        p1 = Jaccard (v, clone_dict[tool][k])
        p2 = subgr (v, clone_dict[tool][k])

        # if p1 > 0.2:
        #     print("\t\tk = {} -> Jaccard Similarity : {}".format ( k, p1 ) )
        #     return False
        if p2 > 0.5:
            print("\t\tk = {} -> Similarity : {}".format ( k, p2 ))
            return False
    return True

In [62]:
import numpy as np
import pandas as pd
import os, subprocess, filetype, re, glob

if __name__ == "__main__":
    INPUT_TSV_DIR = "/data/project/Alzheimer/CLEMENT/01.INPUT_TSV/3.BioData/Brunner_2D/PASS"
    SCRIPT_DIR = "/data/project/Alzheimer/YSscript/cle/"

    from natsort import natsorted
    INPUT_DIR_LIST = natsorted (  glob.glob(INPUT_TSV_DIR + "/PD36714b*") ) 


    toollist = ["CLEMENT_decision", "pyclonevi", "sciclone", "quantumclone", "simpleK_elbow", "simpleK_silhouette", "simpleK_gap"]
    toollist_concise = ["CLEMENT", "pyclonevi", "sciclone", "qc", "simpleK_elb", "simpleK_sil", "simpleK_gap*"]


    INPUT_DIR_LIST_SELECTED = []
    for k in INPUT_DIR_LIST:
        if os.path.isfile (k) == True:
            if "input" in k.split("/")[-1]:
                SAMPLENAME = k.split("/")[-1].split("_input")[0]                
                INPUT_DIR_LIST_SELECTED.append (k)

    INPUT_DIR_LIST = natsorted (  INPUT_DIR_LIST_SELECTED ) 
    
    clone_dict = {}
    for tool in toollist:
        clone_dict [tool] = []

    for INPUT_TSV in INPUT_DIR_LIST:
        kwargs = {"INPUT_TSV" : INPUT_TSV,  "MODE" : "Both",  "NUM_CLONE_TRIAL_START" : 1, "NUM_CLONE_TRIAL_END" : 6, 
                        "TRIAL_NO" : 5, "DEPTH_CUTOFF" : 10,  "KMEANS_CLUSTERNO" : 6, "MIN_CLUSTER_SIZE" : 5,  "MAKEONE_STRICT" :  3,
                        "RANDOM_PICK" : -1, "AXIS_RATIO": -1, "PARENT_RATIO": 0, "NUM_PARENT" : 0,  "FP_RATIO":0,  "FP_USEALL" : "False", 
                        "RANDOM_SEED" : 0, "SAMPLENAME" : "", "BENCHMARK_NO" : 0, 
                        "NPVAF_DIR" : "", "SIMPLE_KMEANS_DIR" : "", "CLEMENT_DIR" : "", "SCICLONE_DIR" : "", "PYCLONEVI_DIR" : "",  "COMBINED_OUTPUT_DIR" : "",
                        "SCORING" : False,  "MAXIMUM_NUM_PARENT" : 2, "VERBOSE" : 1 }
            
        INPUT_FILETYPE, NUM_BLOCK = filetype.main (INPUT_TSV)
        kwargs["NUM_BLOCK_INPUT"] = kwargs["NUM_BLOCK"] = NUM_BLOCK
        SAMPLENAME = kwargs["SAMPLENAME"] = INPUT_TSV.split("/")[-1].split("_input")[0]

        kwargs["NPVAF_DIR"] = "/data/project/Alzheimer/CLEMENT/02.npvaf/3.BioData/Brunner_2D/" + SAMPLENAME
        kwargs["COMBINED_OUTPUT_DIR"] = "/data/project/Alzheimer/CLEMENT/03.combinedoutput/3.BioData/Brunner_2D/" + SAMPLENAME
        kwargs["SIMPLE_KMEANS_DIR"] = "/data/project/Alzheimer/YSscript/cle/data/SIMPLE_KMEANS/3.BioData/Brunner_2D/" + SAMPLENAME
        kwargs["CLEMENT_DIR"] = "/data/project/Alzheimer/YSscript/cle/data/CLEMENT/3.BioData/Brunner_2D/"  + SAMPLENAME
        kwargs["PYCLONEVI_DIR"] = "/data/project/Alzheimer/YSscript/cle/data/pyclone-vi/3.BioData/Brunner_2D/" + SAMPLENAME
        kwargs["SCICLONE_DIR"] = "/data/project/Alzheimer/YSscript/cle/data/sciclone/3.BioData/Brunner_2D/" + SAMPLENAME
        kwargs["QUANTUMCLONE_DIR"] = "/data/project/Alzheimer/YSscript/cle/data/quantumclone/3.BioData/Brunner_2D/" + SAMPLENAME

        print ("{}".format (INPUT_TSV))
        #for tool in toollist:
        for tool in ["CLEMENT_decision"]:
            if not os.path.exists ( kwargs["COMBINED_OUTPUT_DIR"] + "/result/CLEMENT_decision.membership.txt" ):
                continue

            with open(kwargs["COMBINED_OUTPUT_DIR"] + "/result/" + tool + ".membership.txt", "r") as file:
                membership = np.array ( [int(line.strip()) for line in file] ) 
            
                inputdf = pd.read_csv (kwargs["INPUT_TSV"], header = None, sep = "\t")
                variants = list (inputdf.iloc[:, 0])

                print ("\t{}".format (tool))
            
                v = []
                for membership_index in sorted ( list(set(membership)) ) :   # 0 ~ 3
                    print ("\t{} = {}".format (membership_index,  len ( np.where (membership == membership_index) [0] ) ) )
                    v = []
                    for k in np.where (membership == membership_index) [0]:
                        v.append ( variants [k] )

                    if add_v_clone ( tool, clone_dict, v ) == True:
                        clone_dict [tool].append (v)
                    #print (v)




print (clone_dict)


        


/data/project/Alzheimer/CLEMENT/01.INPUT_TSV/3.BioData/Brunner_2D/PASS/PD36714b_lo013_PD36714b_lo025_input.txt
	CLEMENT_decision
	0 = 654
	1 = 233
	2 = 384
/data/project/Alzheimer/CLEMENT/01.INPUT_TSV/3.BioData/Brunner_2D/PASS/PD36714b_lo013_PD36714b_lo038_input.txt
	CLEMENT_decision
	0 = 375
		k = 0 -> Similarity : 0.856
	1 = 1172
		k = 2 -> Similarity : 0.6432291666666666
	2 = 512
		k = 0 -> Similarity : 0.650390625
/data/project/Alzheimer/CLEMENT/01.INPUT_TSV/3.BioData/Brunner_2D/PASS/PD36714b_lo013_PD36714b_lo047_input.txt
	CLEMENT_decision
	0 = 398
		k = 0 -> Similarity : 0.8417085427135679
	1 = 489
		k = 0 -> Similarity : 0.6523517382413088
	2 = 858
		k = 2 -> Similarity : 0.5989583333333334
/data/project/Alzheimer/CLEMENT/01.INPUT_TSV/3.BioData/Brunner_2D/PASS/PD36714b_lo025_PD36714b_lo038_input.txt
	CLEMENT_decision
	0 = 191
		k = 2 -> Similarity : 0.7172774869109948
	1 = 1258
		k = 0 -> Similarity : 0.5091743119266054
	2 = 426
		k = 1 -> Similarity : 0.7682403433476395
/data/p

In [39]:
for key in clone_dict.keys():
    print ("{} = {}". format (key , len ( clone_dict[key]) ) )

CLEMENT_decision = 4
pyclonevi = 8
sciclone = 5
quantumclone = 7
simpleK_elbow = 3
simpleK_silhouette = 5
simpleK_gap = 6
